# Project Start Here

In [148]:
import requests as re
import pandas as pd
import numpy as np
import json
import math
from pandas.io.json import json_normalize

## get the data
---
- Get the data by api.
- Save the data as 'raw.csv', which is the raw data ready for data preporcessing.
---
Totaly 2,000 pics of data sorted by time from resent to past.

In [169]:
def get_data():
    zed_index = '?'
    include_sold = '1'
    page_number = 1
    listing_status = 'sale'
    page_size = '100'
    order_by = 'age'
    area = 'somerset'
    api_key = 'jwttfkz79asz2d9h8sms7tuu'
    for page_number in range(1, 59):
        url = f"http://api.zoopla.co.uk/api/v1/property_listings.js?zed_index=?&include_sold=1&page_number={page_number}&listing_status=sale&page_size=100&order_by=age&area=somerset&api_key=jwttfkz79asz2d9h8sms7tuu"
        response = re.get(url)
        df = json_normalize(response.json()['listing'])
        if page_number == 1:
            df.to_csv('raw.csv', index = False)
        else:
            df.to_csv('raw.csv', index = False, mode = 'a', header=False)

In [170]:
## Load Raw Data
get_data()

In [171]:
raw = pd.read_csv('raw.csv')

ParserError: Error tokenizing data. C error: Expected 43 fields in line 102, saw 45


In [168]:
raw

,agent_address,agent_logo,agent_name,agent_phone,category,country,country_code,county,description,details_url,...,price_change,price_change_summary.direction,price_change_summary.last_updated_date,price_change_summary.percent,price_modifier,property_type,short_description,status,street_name,thumbnail_url
0,"15 Sadler Street, Wells",https://st.zoocdn.com/zoopla_static_agent_logo...,Allen & Harris - Wells,01749 681031,Residential,England,gb,Somerset,SummaryA Charming period property located in t...,https://www.zoopla.co.uk/for-sale/details/4712...,...,"[{'direction': '', 'date': '2018-03-30 15:08:3...",down,2018-12-26 06:22:36,-8.8%,NaN,Flat,A Charming period property located in the cent...,for_sale,Chamberlain Street,https://lid.zoocdn.com/80/60/d29960dc0453e5663...
1,"Park 5 Park Road, Yeovil",https://st.zoocdn.com/zoopla_static_agent_logo...,Evolve,01935 590148,Residential,England,gb,Somerset,Detailed DescriptionA beautifully presented Gr...,https://www.zoopla.co.uk/for-sale/details/4710...,...,"[{'direction': '', 'date': '2018-03-29 15:23:0...",down,2019-01-04 09:28:44,-8.6%,NaN,Flat,A beautifully presented Ground Floor apartment...,for_sale,Wyndham Court,https://lid.zoocdn.com/80/60/0640ec51127c97a2f...
2,"16 Princes Street, Yeovil",https://st.zoocdn.com/zoopla_static_agent_logo...,Palmer Snell - Yeovil Sales,01935 590853,Residential,England,gb,Somerset,"This beautifully presented, three bedroom, Ham...",https://www.zoopla.co.uk/for-sale/details/4710...,...,"[{'direction': '', 'date': '2018-03-29 11:39:2...",down,2019-02-25 12:39:29,-8.3%,offers_over,End terrace house,"This beautifully presented, three bedroom, Ham...",sold,Stoke-Sub-Hamdon Somerset,https://lid.zoocdn.com/80/60/c6c0d656b4e6526d7...
3,"Corporation Street, Taunton",https://st.zoocdn.com/zoopla_static_agent_logo...,Robert Cooney,01823 760884,Residential,England,gb,Somerset,Requiring modernisation and improvement and on...,https://www.zoopla.co.uk/for-sale/details/4707...,...,"[{'direction': '', 'date': '2018-03-27 16:29:1...",down,2019-03-25 16:41:47,-17.6%,NaN,Detached bungalow,"<p class=""top"">Requiring modernisation and imp...",sold,Green Lane,https://lid.zoocdn.com/80/60/f5097a235d586cbb2...
4,"Corporation Street, Taunton",https://st.zoocdn.com/zoopla_static_agent_logo...,Robert Cooney,01823 760884,Residential,England,gb,Somerset,A rare opportunity to acquire a classic 1930's...,https://www.zoopla.co.uk/for-sale/details/4707...,...,"[{'direction': '', 'date': '2018-03-27 16:29:1...",down,2019-02-16 11:18:46,-20%,offers_over,Detached house,"<p class=""top"">A rare opportunity to acquire a...",sold,Mount Nebo,https://lid.zoocdn.com/80/60/3bc0a48e8218139e3...
5,"Corporation Street, Taunton",https://st.zoocdn.com/zoopla_static_agent_logo...,Robert Cooney,01823 760884,Residential,England,gb,Somerset,A spacious 3 double bedroomed South facing det...,https://www.zoopla.co.uk/for-sale/details/4707...,...,"[{'direction': '', 'date': '2018-03-27 16:29:0...",down,2018-07-04 16:05:40,-5.9%,NaN,Detached bungalow,"<p class=""top"">A spacious 3 double bedroomed S...",for_sale,Sweethay Close,https://lid.zoocdn.com/80/60/dba9162e4bde21f1a...
6,"Corporation Street, Taunton",https://st.zoocdn.com/zoopla_static_agent_logo...,Robert Cooney,01823 760884,Residential,England,gb,Somerset,In a quiet cul de sac location in this popular...,https://www.zoopla.co.uk/for-sale/details/4707...,...,"[{'direction': '', 'date': '2018-03-27 16:29:1...",down,2018-08-15 08:33:08,-5%,offers_over,Detached house,"<p class=""top"">In a quiet cul de sac location ...",for_sale,Lydeard Mead,https://lid.zoocdn.com/80/60/baf196278240cc07e...
7,"Corporation Street, Taunton",https://st.zoocdn.com/zoopla_static_agent_logo...,Robert Cooney,01823 760884,Residential,England,gb,Somerset,A light studio apartment with large picture wi...,https://www.zoopla.co.uk/for-sale/details/4707...,...,"[{'direction': '', 'date': '2018-03-27 16:29:0...",down,2018-07-03 12:17:56,-6.6%,NaN,Studio,"<p class=""top"">A light st

### For get data testing
---
Testing the api and csv save method

In [116]:
zed_index = '?'
include_sold = '1'
page_number = 3
listing_status = 'sale'
page_size = '20'
order_by = 'age'
area = 'somerset'
api_key = 'jwttfkz79asz2d9h8sms7tuu'

url = 'http://api.zoopla.co.uk/api/v1/property_listings.js?zed_index=?&include_sold=1&page_number=58&listing_status=sale&page_size=100&order_by=age&area=somerset&api_key=jwttfkz79asz2d9h8sms7tuu'
response = re.get(url)
df = json_normalize(response.json()['listing'])
df['displayable_address']

0                         Silver Street, Ilminster TA19
1                            Quay Street, Minehead TA24
2     Beach Road, Weston Super Mare, North Somerset ...
3                Littlewell Cottages, Coxley, Wells BA5
4                 Maunsell Road, Weston-Super-Mare BS24
5                              Silver Street, Wells BA5
6                 Clevedon Road, Weston-Super-Mare BS23
7                           Walton Court, Minehead TA24
8                Preston Bowyer, Milverton, Taunton TA4
9                 Maunsell Road, Weston-Super-Mare BS24
10           Castle Hill, Nether Stowey, Bridgwater TA5
11           Castle Hill, Nether Stowey, Bridgwater TA5
12                            Bow Street, Langport TA10
13                                    Wookey, Wells BA5
14    Locking Road, Weston-Super-Mare, North Somerse...
15    Upper Church Road, Weston-Super-Mare, North So...
16                         Burrowbridge, Bridgwater TA7
17                          Western Lane, Minehe

# Data Preporcessing

In [117]:
raw.columns

Index(['agent_address', 'agent_logo', 'agent_name', 'agent_phone', 'category',
       'country', 'country_code', 'county', 'description', 'details_url',
       'displayable_address', 'first_published_date', 'floor_plan',
       'furnished_state', 'image_150_113_url', 'image_354_255_url',
       'image_50_38_url', 'image_645_430_url', 'image_80_60_url',
       'image_caption', 'image_url', 'last_published_date', 'latitude',
       'letting_fees', 'listing_id', 'listing_status',
       'location_is_approximate', 'longitude', 'new_home', 'num_bathrooms',
       'num_bedrooms', 'num_floors', 'num_recepts', 'outcode', 'post_town',
       'price', 'price_change', 'price_modifier', 'property_type',
       'short_description', 'status', 'street_name', 'thumbnail_url'],
      dtype='object')

In [156]:
useful_information = raw[['latitude', 'longitude', 'property_type','category', 'description', 'num_bathrooms', 'num_bedrooms','num_floors', 'num_recepts', 'outcode', 'post_town', 'street_name', 'new_home', 'price']].reset_index()

In [164]:
hot_place = useful_information[['latitude', 'longitude', 'price']]
hot_place.to_csv('hot_place.csv', index = False)
hot_place

,latitude,longitude,price
0,51.026470,-3.103359,120000
1,51.270325,-2.775121,370000
2,50.981876,-3.226012,200000
3,51.360435,-2.938179,219950
4,51.318280,-2.809450,659000
5,50.877384,-2.945940,350000
6,50.956654,-2.603365,170000
7,50.947227,-2.622901,160000
8,50.940390,-2.634381,150000
9,51.032870,-3.088850,169950


In [157]:
def new_home(value):
    if math.isnan(value):
        return 0
    else:
        return 1
    
useful_information['new_home'] = useful_information['new_home'].apply(new_home)

In [158]:
useful_information

,index,latitude,longitude,property_type,category,description,num_bathrooms,num_bedrooms,num_floors,num_recepts,outcode,post_town,street_name,new_home,price
0,0,51.026470,-3.103359,Flat,Residential,"A spacious ground floor flat, available with n...",1,1,0,1,TA2,Taunton,Portman Street,0,120000
1,1,51.270325,-2.775121,Detached house,Residential,A good size detached house set on this sought ...,0,4,0,0,BS27,Cheddar,Charterhouse Close,0,370000
2,2,50.981876,-3.226012,Terraced house,Residential,This is a lovely spacious 3 bedroom terraced h...,1,3,0,2,TA21,Wellington,Drakes Park,0,200000
3,3,51.360435,-2.938179,Detached bungalow,Residential,One off design Detached Bungalow with accommod...,0,2,0,0,BS22,Weston-super-Mare,Whittington Drive,0,219950
4,4,51.318280,-2.809450,Detached house,Residential,Individual Detached HouseApprox 1/2 Acre of Gr...,0,4,0,0,BS25,Winscombe,Horseleaze Lane,0,659000
5,5,50.877384,-2.945940,Detached house,Residential,An extremely well presented detached 4 bedroom...,2,4,0,2,TA20,Chard,Caraway Close,0,350000
6,6,50.956654,-2.603365,End terrace house,Residential,Detailed DescriptionHidden away this two doubl...,2,2,0,1,BA21,Yeovil,Great Mead,0,170000
7,7,50.947227,-2.622901,Terraced house,Residential,Detailed DescriptionEvolve estate agents offer...,1,3,0,2,BA21,Yeovil,Grass Royal,0,160000
8,8,50.940390,-2.634381,Flat,Residential,Detailed DescriptionPositioned in the heart of...,1,3,0,1,BA20,Yeovil,Flowers House,0,150000
9,9,51.032870,-3.088850,Semi-detached house,Residential,This lovely two double bedroom Semi-Detached h...,1,2,0,1,TA2,Taunton,Buckland Road,0,169950


In [126]:
cate = useful_information.drop_duplicates(['category'])
cate

,index,latitude,longitude,property_type,category,description,num_bathrooms,num_bedrooms,num_floors,num_recepts,outcode,post_town,street_name,new_home,price
0,0,51.02647,-3.103359,Flat,Residential,"A spacious ground floor flat, available with n...",1,1,0,1,TA2,Taunton,Portman Street,NaN,120000


In [127]:
property_type = useful_information.drop_duplicates(['property_type'])
property_type['property_type']

0                       Flat
1             Detached house
2             Terraced house
3          Detached bungalow
6          End terrace house
9        Semi-detached house
10                   Cottage
14                       NaN
15                  Bungalow
16    Semi-detached bungalow
22       Link-detached house
40                      Land
56                    Studio
61             Country house
70                     Lodge
89                 Farmhouse
Name: property_type, dtype: object

In [128]:
code = useful_information.drop_duplicates(['outcode'])
code['outcode']

0      TA2
1     BS27
2     TA21
3     BS22
4     BS25
5     TA20
6     BA21
8     BA20
10     TA6
12    BS21
16     BA6
17     BA5
18     TA1
20     TA5
26    BS24
27     SP8
28    BA11
29     TA3
32    TA12
37    TA13
38    BA22
40    TA18
42    BS40
45    TA10
47     BA4
55     BA9
56     TA8
60    BS23
69    TA24
78     TA4
81     TA9
82    BA16
88    BS49
Name: outcode, dtype: object

In [129]:
post_town = useful_information.drop_duplicates(['post_town'])
post_town['post_town']

0               Taunton
1               Cheddar
2            Wellington
3     Weston-super-Mare
4             Winscombe
5                 Chard
6                Yeovil
10           Bridgwater
12             Clevedon
16          Glastonbury
17                Wells
27           Gillingham
28                Frome
32              Martock
37      South Petherton
40            Crewkerne
42              Bristol
45             Langport
47       Shepton Mallet
55            Wincanton
56       Burnham-on-Sea
69             Minehead
81           Highbridge
82               Street
Name: post_town, dtype: object

In [130]:
street_name = useful_information.drop_duplicates(['street_name'])
street_name['street_name']

0                                Portman Street
1                            Charterhouse Close
2                                   Drakes Park
3                             Whittington Drive
4                               Horseleaze Lane
5                                 Caraway Close
6                                    Great Mead
7                                   Grass Royal
8                                 Flowers House
9                                 Buckland Road
10                               Chilton Street
11                                Condell Close
12                                    Kenn Road
13                              Loxleigh Avenue
14                                  Gough Place
15                                Langdon Close
16                                Ferryman Road
17                                Alfred Street
18                                 South Street
19                                  Dragon Rise
20                               Barberr

In [159]:
## one-hot encode
useful_information = pd.get_dummies(useful_information)

In [160]:
useful_information['new_home']

0       0
1       0
2       0
3       0
4       0
5       0
6       0
7       0
8       0
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      0
17      0
18      0
19      0
20      0
21      0
22      0
23      1
24      1
25      1
26      0
27      0
28      0
29      0
       ..
5770    0
5771    0
5772    0
5773    0
5774    0
5775    0
5776    0
5777    1
5778    0
5779    0
5780    0
5781    0
5782    0
5783    0
5784    0
5785    0
5786    0
5787    0
5788    1
5789    0
5790    0
5791    0
5792    0
5793    0
5794    0
5795    0
5796    0
5797    0
5798    0
5799    0
Name: new_home, Length: 5800, dtype: int64

In [172]:
useful_information

,index,latitude,longitude,num_bathrooms,num_bedrooms,num_floors,num_recepts,new_home,price,property_type_Bungalow,...,street_name_Tower Road,street_name_Union Street,street_name_Victoria Court,street_name_Warminster Road,street_name_Weston-Super-Mare North Somerset,street_name_Whittington Drive,street_name_Willow Gardens St. Georges,street_name_Woodpecker Drive Worle,street_name_Wyndham Court,street_name_Yeovil Somerset
0,0,51.026470,-3.103359,1,1,0,1,0,120000,0,...,0,0,0,0,0,0,0,0,0,0
1,1,51.270325,-2.775121,0,4,0,0,0,370000,0,...,0,0,0,0,0,0,0,0,0,0
2,2,50.981876,-3.226012,1,3,0,2,0,200000,0,...,0,0,0,0,0,0,0,0,0,0
3,3,51.360435,-2.938179,0,2,0,0,0,219950,0,...,0,0,0,0,0,1,0,0,0,0
4,4,51.318280,-2.809450,0,4,0,0,0,659000,0,...,0,0,0,0,0,0,0,0,0,0
5,5,50.877384,-2.945940,2,4,0,2,0,350000,0,...,0,0,0,0,0,0,0,0,0,0
6,6,50.956654,-2.603365,2,2,0,1,0,170000,0,...,0,0,0,0,0,0,0,0,0,0
7,7,50.947227,-2.622901,1,3,0,2,0,160000,0,...,0,0,0,0,0,0,0,0,0,0
8,8,50.940390,-2.634381,1,3,0,1,0,150000,0,...,0,0,0,0,0,0,0,0,0,0
9,9,51.032870,-3.088850,1,2,0,1,0,169950,0,...,0,0,0,0,0,0,0,0,0,0
